## 1. Import Modules and Data

In [1]:
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("load tokenizer")

load tokenizer


## 2. Load Trained Model

In [2]:
import torch
from modules.bert import BERTTextClassifier
import config 

bert_clf = BERTTextClassifier.from_pretrained(
    config.pretrained_path,  # 配置文件中的预训练模型目录
    num_frozen_layers=12,  # 冻结层
)
bert_clf.load_state_dict(torch.load(config.checkpoint_dir / "bert_clf_10.pth")["model"])
print("load done!")

Number of trainable parameters: 108.50M
load done!


## 3. Inference


Here I consider text classification task with simple example as below.

In [7]:
text = "I am extremely disappointed with the quality of this item."

input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)

predicted_class = bert_clf.text_clf(input_ids)

print(f"Predicted class index: {predicted_class.item()}")

text_clf_logits:tensor([[-0.8150, -0.0207]])
Predicted class index: 1
